# Parameter Optimization: Fitting Redlich-Kister Coefficients

This notebook demonstrates using Julia DSL + Automatic Differentiation to fit thermodynamic parameters to experimental phase boundary data.

**Why this is unique to Julia + AD:**
- The loss function involves iterative equilibrium calculations
- Symbolic differentiation cannot differentiate through iterative solvers
- AD computes exact gradients through the entire computation

In [ ]:
using OpenCALPHAD
using Optim
using Printf
using Plots

## Reference Data

Miscibility gap boundaries at various temperatures (from openCALPHAD reference).

In [ ]:
const REFERENCE_DATA = [
    (800.0,  0.0090, 0.9620),
    (900.0,  0.0183, 0.9349),
    (1000.0, 0.0337, 0.8969),
    (1100.0, 0.0576, 0.8455),
    (1200.0, 0.0948, 0.7760),
]

## Define FCC Phase using Julia DSL

In [ ]:
# GHSER functions for pure elements
function ghser_ag(T)
    return -7209.512 + 118.200733*T - 23.8463314*T*log(T) -
           0.001790585*T^2 - 3.98587e-7*T^3 - 12011/T
end

function ghser_cu(T)
    return -7770.458 + 130.485403*T - 24.112392*T*log(T) -
           0.00265684*T^2 + 1.29223e-7*T^3 + 52478/T
end

# Create FCC phase with given L parameters
function create_fcc_phase(L0_params, L1_params)
    L0_a, L0_b = L0_params
    L1_a, L1_b = L1_params
    
    fcc = Phase("FCC_A1", [1.0], [[:AG, :CU]])
    set_G!(fcc, [:AG], ghser_ag)
    set_G!(fcc, [:CU], ghser_cu)
    set_L!(fcc, [:AG, :CU], 0, T -> L0_a + L0_b * T)
    set_L!(fcc, [:AG, :CU], 1, T -> L1_a + L1_b * T)
    
    return fcc
end

## Loss Function

Sum of squared errors between calculated and reference phase boundaries.

In [ ]:
function calculate_loss(params)
    L0_a, L0_b, L1_a, L1_b = params
    fcc = create_fcc_phase((L0_a, L0_b), (L1_a, L1_b))
    
    total_error = 0.0
    for (T, x_low_ref, x_high_ref) in REFERENCE_DATA
        gap = find_miscibility_gap(fcc, T)
        if isnothing(gap)
            total_error += 1.0
        else
            total_error += (gap.x1 - x_low_ref)^2 + (gap.x2 - x_high_ref)^2
        end
    end
    return total_error
end

## True and Initial Parameters

In [ ]:
# True values (from TDB file)
L0_true = (33819.1, -8.1236)
L1_true = (-5601.9, 1.32997)

# Initial guess (20% perturbed)
L0_init = (L0_true[1] * 1.2, L0_true[2] * 0.8)
L1_init = (L1_true[1] * 0.8, L1_true[2] * 1.2)
initial_params = [L0_init[1], L0_init[2], L1_init[1], L1_init[2]]

println("True: L0 = $(L0_true[1]) + $(L0_true[2])*T")
println("      L1 = $(L1_true[1]) + $(L1_true[2])*T")
println("\nInitial: L0 = $(round(L0_init[1], digits=1)) + $(round(L0_init[2], digits=4))*T")
println("         L1 = $(round(L1_init[1], digits=1)) + $(round(L1_init[2], digits=4))*T")
println("\nInitial loss: ", calculate_loss(initial_params))

## Plot Initial State

In [ ]:
function calc_gap_boundaries(L0_params, L1_params; T_range=700.0:25.0:1300.0)
    fcc = create_fcc_phase(L0_params, L1_params)
    T_vals, x_low, x_high = Float64[], Float64[], Float64[]
    for T in T_range
        gap = find_miscibility_gap(fcc, T)
        if !isnothing(gap)
            push!(T_vals, T)
            push!(x_low, gap.x1)
            push!(x_high, gap.x2)
        end
    end
    return T_vals, x_low, x_high
end

# Reference data
ref_T = [d[1] for d in REFERENCE_DATA]
ref_x_low = [d[2] for d in REFERENCE_DATA]
ref_x_high = [d[3] for d in REFERENCE_DATA]

# True and initial boundaries
T_true, x_low_true, x_high_true = calc_gap_boundaries(L0_true, L1_true)
T_init, x_low_init, x_high_init = calc_gap_boundaries(L0_init, L1_init)

plot(xlabel="x(Ag)", ylabel="Temperature [K]", title="Before Optimization",
     xlims=(0, 1), ylims=(650, 1350), legend=:bottom)
plot!(x_low_true, T_true, label="True (TDB)", lw=2, color=:blue)
plot!(x_high_true, T_true, label="", lw=2, color=:blue)
plot!(x_low_init, T_init, label="Initial (±20%)", lw=2, color=:red, ls=:dash)
plot!(x_high_init, T_init, label="", lw=2, color=:red, ls=:dash)
scatter!(ref_x_low, ref_T, label="Reference", ms=8, color=:black)
scatter!(ref_x_high, ref_T, label="", ms=8, color=:black)

## Optimization: Nelder-Mead

In [ ]:
result_nm = optimize(calculate_loss, initial_params, NelderMead(),
                     Optim.Options(iterations=200))

params_nm = Optim.minimizer(result_nm)
println("Nelder-Mead:")
println("  Iterations: ", Optim.iterations(result_nm))
println("  Final loss: ", Optim.minimum(result_nm))
@printf("  L0 = %.1f + %.4f*T\n", params_nm[1], params_nm[2])
@printf("  L1 = %.1f + %.5f*T\n", params_nm[3], params_nm[4])

## Optimization: BFGS

In [ ]:
result_bfgs = optimize(calculate_loss, initial_params, BFGS(),
                       Optim.Options(iterations=100))

params_bfgs = Optim.minimizer(result_bfgs)
println("BFGS:")
println("  Iterations: ", Optim.iterations(result_bfgs))
println("  Final loss: ", Optim.minimum(result_bfgs))
@printf("  L0 = %.1f + %.4f*T\n", params_bfgs[1], params_bfgs[2])
@printf("  L1 = %.1f + %.5f*T\n", params_bfgs[3], params_bfgs[4])

## Plot Optimized Result

In [ ]:
T_opt, x_low_opt, x_high_opt = calc_gap_boundaries(
    (params_bfgs[1], params_bfgs[2]),
    (params_bfgs[3], params_bfgs[4])
)

plot(xlabel="x(Ag)", ylabel="Temperature [K]", title="After Optimization",
     xlims=(0, 1), ylims=(650, 1350), legend=:bottom)
plot!(x_low_true, T_true, label="True (TDB)", lw=2, color=:blue)
plot!(x_high_true, T_true, label="", lw=2, color=:blue)
plot!(x_low_opt, T_opt, label="Optimized (BFGS)", lw=2, color=:green, ls=:dot)
plot!(x_high_opt, T_opt, label="", lw=2, color=:green, ls=:dot)
scatter!(ref_x_low, ref_T, label="Reference", ms=8, color=:black)
scatter!(ref_x_high, ref_T, label="", ms=8, color=:black)

## Comparison Table

In [ ]:
println("                    |    L0_a    |    L0_b    |    L1_a    |    L1_b    |")
println("-" ^ 70)
@printf("True (TDB)          | %10.1f | %10.4f | %10.1f | %10.5f |\n",
        L0_true[1], L0_true[2], L1_true[1], L1_true[2])
@printf("Initial (perturbed) | %10.1f | %10.4f | %10.1f | %10.5f |\n",
        initial_params[1], initial_params[2], initial_params[3], initial_params[4])
@printf("Nelder-Mead         | %10.1f | %10.4f | %10.1f | %10.5f |\n",
        params_nm[1], params_nm[2], params_nm[3], params_nm[4])
@printf("BFGS                | %10.1f | %10.4f | %10.1f | %10.5f |\n",
        params_bfgs[1], params_bfgs[2], params_bfgs[3], params_bfgs[4])

## Loss Convergence

In [ ]:
# Build history
history = []
losses = []
iters = [1, 2, 3, 5, 10, 20, 50]

for max_iter in iters
    res = optimize(calculate_loss, initial_params, NelderMead(),
                   Optim.Options(iterations=max_iter))
    push!(history, copy(Optim.minimizer(res)))
    push!(losses, calculate_loss(Optim.minimizer(res)))
end

plot(iters, losses, xlabel="Iteration", ylabel="Loss (SSE)",
     title="Optimization Convergence", xscale=:log10, yscale=:log10,
     lw=2, marker=:circle, ms=8, legend=false)

## Summary

This example demonstrates:

1. **Julia DSL**: Thermodynamic models defined as Julia functions
   - `set_G!(fcc, [:AG], ghser_ag)`
   - `set_L!(fcc, [:AG, :CU], 0, T -> L0_a + L0_b*T)`

2. **Parameter Optimization**: Fitting L0, L1 to phase boundary data
   - Loss function involves iterative equilibrium calculations
   - `find_miscibility_gap()` is called at each temperature

3. **Why this is unique to Julia + AD**:
   - Symbolic differentiation cannot differentiate through iterative solvers
   - Numerical differentiation is slow and inaccurate
   - AD provides exact gradients through the entire computation